In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import mlflow

## Question 1

In [3]:
!pip list | grep mlflow
!mlflow --version

mlflow                    1.26.0
mlflow, version 1.26.0


## Q1: Version 1.26.0

## Question 2

In [7]:
!python ./homework/preprocess_data.py --raw_data_path '~/data/' --dest_path ./output

In [9]:
!ls -l ./output/

total 7756
-rw-rw-r-- 1 ubuntu ubuntu  305256 May 26 19:59 dv.pkl
-rw-rw-r-- 1 ubuntu ubuntu 2805197 May 26 19:59 test.pkl
-rw-rw-r-- 1 ubuntu ubuntu 2661149 May 26 19:59 train.pkl
-rw-rw-r-- 1 ubuntu ubuntu 2166527 May 26 19:59 valid.pkl


## Q2: 4 files

## Question 3

In [19]:
!python ./homework/train.py --data_path ./output

2022/05/26 20:16:14 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/26 20:16:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [15]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='nyc-taxi-experiment-homework', tags={}>]

In [21]:
runs = client.search_runs(
    experiment_ids='3',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}")

run id: 6200c1a44de947ff85c3accc9ba3cb47
run id: c08180a70d534e8483f3bd36c264f8db


In [26]:
params = len(runs[-1].data.params)
print(params)

17


## Q3: 17 Parameters

## Question 4

In [28]:
! mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts/

[2022-05-26 20:23:04 +0000] [3155] [INFO] Starting gunicorn 20.1.0
[2022-05-26 20:23:04 +0000] [3155] [INFO] Listening at: http://127.0.0.1:5000 (3155)
[2022-05-26 20:23:04 +0000] [3155] [INFO] Using worker: sync
[2022-05-26 20:23:05 +0000] [3156] [INFO] Booting worker with pid: 3156
^C
[2022-05-26 20:23:34 +0000] [3155] [INFO] Handling signal: int
[2022-05-26 20:23:34 +0000] [3156] [INFO] Worker exiting (pid: 3156)


_I know it would be silly to actually run this in here I just wanted to demonstrate haha_

## Q4: Just --default-artifact-root <URI>

## Q5 Hyperparameter tuning

In [29]:
!python ./homework/hpo.py --data_path ./output

2022/05/26 20:28:09 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|█████████| 50/50 [08:52<00:00, 10.65s/trial, best loss: 6.6284257482044735]


## Q5: Best validation RMSE is 6.628

## Question 6 

In [33]:
!python ./homework/register_model.py --data_path ./output

2022/05/26 20:58:29 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/26 20:58:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."



run id: 045220366dbb40318408a468ec69641d, rmse: 6.5479



Successfully registered model 'random-forest-best-models'.
2022/05/26 21:00:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: random-forest-best-models, version 1
Created version '1' of model 'random-forest-best-models'.


run id: 72342164679d4f339142015dab5d280f, rmse: 6.5479

## Q6: ~ 6.55